In [ ]:
from treelib import Node, Tree
import pandas as pd

## Examples with the treelib library
---

In [ ]:
# Example Tree
tree = Tree()
tree.create_node("Ryan","ryan")
tree.create_node("n1","n1", parent="ryan")
tree.create_node("n2","n2", parent="ryan")
tree.show()

In [ ]:
# Check if a node exists using .get_node() method
tree.get_node("ryan")

In [ ]:
# Check if no node exists with the 
tree.get_node("macy")

### Data ingestion / conversion
Read in the custom metrics .csv file and create a dataframe to run into analysis

In [ ]:
def read_custom_metrics_csv(fn:str) -> pd.DataFrame():
    """
    Purpose: Read in the custom metrics .csv file and make 
    small transformations to it. 
    
    Input Args: 
        - fn(str): The filepath to the .csv
    Output: 
        - pd.DataFrame: Transformed custom metrics data
    """
    df = pd.read_csv(fn)
    df = df.rename(columns={
        "Metric Name":"metric_name",
        "Average Custom Metrics / Hour": "average_hourly_custom_metrics",
        "Max Custom Metrics / Hour": "max_hourly_custom_metrics"
        })
    return df

In [ ]:
cm_df = read_custom_metrics_csv(fn="./test-data/example-custom-metrics.csv")
cm_df.head()

## Converting Custom Metrics List into Tree
----

### Creating the Tree Constructor

Testing with 3 different lists to see if the tree gets built out appropriately

In [ ]:
metric_list1 = ["mongo","cache","performance","cpu"]
metric_list2 = ["mongo","cache","performance","ram"]
metrics_list3 = ["redis","cache","performance","cpu"]

In [ ]:
def add_metric_namespace_to_tree(metric_list:list, tree:Tree) -> Tree:
    parent="root"
    for element in metric_list:
        node_id = parent + "." + element
        # Check if the element exists in the parent's subtree
        # If it does skip, o.w. add to the tree
        if not tree.subtree(parent).get_node(node_id):
            tree.create_node(element, node_id, parent=parent)
        parent = node_id
    return tree
    

In [ ]:
# Initiate the tree
cm_tree = Tree()
cm_tree.create_node("root","root")

# Test with two metrics lists
cm_tree = add_metric_namespace_to_tree(metric_list=metric_list1, tree=cm_tree)
cm_tree = add_metric_namespace_to_tree(metric_list=metric_list2, tree=cm_tree)
cm_tree = add_metric_namespace_to_tree(metric_list=metrics_list3, tree=cm_tree)
cm_tree.show()

### Putting it all together

In [ ]:
def generate_cm_namespace_tree(cm_df:pd.DataFrame) -> Tree: 
    cm_tree = Tree()
    cm_tree.create_node("root","root")
    for iter, row in cm_df.iterrows():
        custom_metric = row.metric_name
        metric_namespace = custom_metric.split(".")
        cm_tree = add_metric_namespace_to_tree(metric_namespace,cm_tree)
    return cm_tree

In [ ]:
cm_df = read_custom_metrics_csv("./test-data/example-custom-metrics.csv")
cm_tree = generate_cm_namespace_tree(cm_df)
cm_tree.show()